# Liver Model Construction: Glycolysis Gluconeogenesis & Lactate Fermentation

## Setup workflow

### Import packages

In [10]:
import os
import warnings
from cobra.io.json import load_json_model as load_json_cobra_model
import escher
import mass
import numpy as np
import pandas as pd
import sympy as sy
from cobra import Model, Reaction, Metabolite
import cobra.test
from os.path import join
from mass import MassMetabolite, MassModel, MassReaction
from mass.io.json import save_json_model as save_json_mass_model
from mass.visualization import plot_comparison, plot_time_profile

print("MASSpy version: {0}".format(mass.__version__))

MASSpy version: 0.1.1


## Set paths and constants

### Directory paths

In [11]:
model_dir = os.path.abspath("../models")
maps_dir = os.path.abspath("../maps")

# Create MASS model from COBRA model

### Load COBRA model

In [12]:
core_model=load_json_cobra_model(filename=os.path.join(model_dir,"CoreModel.json"))
core_model

Name,None
Memory address,0x07fec25670510
Number of metabolites,178
Number of reactions,187
Number of groups,0
Objective expression,1.0*ATPM - 1.0*ATPM_reverse_5b752
Compartments,"m, i, c, r,"


## Create MASS model

In [13]:
glycolysis = MassModel("Glycolysis", array_type='DataFrame', dtype=np.int64)

### Define reactions

In [14]:
# glycolysis_map = escher.Builder(model=core_model,  map_json="Glycolysis, gluconeogenesis lactate fermentation 2.json",
#                                 highlight_missing=True) 
# glycolysis_map

In [15]:
reaction_list = ['HEX1',
                'PGI',
#                 'FBP',
                'PFK',
                'FBA',
                'TPI',
                'GAPD',
                'PGK',
                'PGM',
                'ENO',
#                 'PEPtm',
#                 'PEPCKm',
                 'LDH_L',
                'PYK']
#                 'PCm',
                 
#                  'PYRt2m']
new_reaction_list = []
for item in reaction_list: 
    item = core_model.reactions.get_by_id(item)
    new_reaction_list.append(item) 
    
new_reaction_list

# Convert cobra.Reactions to mass.MassReactions
for rid in reaction_list:
    reaction = core_model.reactions.get_by_id(rid)
    glycolysis.add_reactions([MassReaction(reaction)])
    
## set possible exchanges for in and out?




#### adding from masspy documentation

In [20]:
m=["adp_c","amp_c", "atp_c","pi_c","nadh_c","nad_c", "h2o_c", "h_c"]
new_list = []
for item in m: 
    item = core_model.metabolites.get_by_id(item)
    new_list.append(item) 
    
# new_reaction_list

# Convert cobra.Reactions to mass.MassReactions
for rid in m:
    met = core_model.metabolites.get_by_id(rid)
    glycolysis.add_metabolites([MassMetabolite(met)])


# # Add PFK1 activator gdp 
# glycolysis.add_metabolites([MassMetabolite(core_model.metabolites.gdp_c)])
# Set concentrations of hydrogen, water, and GDP as fixed
# for metabolite in ["adp_c","amp_c", "atp_c","pi_c","nadh_c","nad_c", "h2o_c", "h_c"]:
#     metabolite = glycolysis.metabolites.get_by_id(metabolite)
#     metabolite.fixed = True
ADK1 = MassReaction(
    "ADK1",
    name="Adenylate kinase",
    subsystem="Misc.",
    reversible=True)
ADK1.add_metabolites({
    adp_c: -2,
    amp_c: 1,
    atp_c: 1})

ATPM = MassReaction(
    "ATPM",
    name="ATP maintenance requirement",
    subsystem="Pseudoreaction",
    reversible=False)
ATPM.add_metabolites({
    atp_c: -1,
    h2o_c: -1,
    adp_c: 1,
    h_c: 1,
    pi_c: 1})

DM_nadh = MassReaction(
    "DM_nadh",
    name="Demand NADH",
    subsystem="Pseudoreaction",
    reversible=False)
DM_nadh.add_metabolites({
    nadh_c: -1,
    nad_c: 1,
    h_c: 1})
glycolysis.add_reactions([
     ADK1, ATPM, DM_nadh])

NameError: name 'adp_c' is not defined

### View extracted network

In [15]:
escher_builder = escher.Builder(
    model=glycolysis,
    map_json=os.path.join(
        maps_dir, ".".join((
            glycolysis.id, "map", "json"))
    ),
    highlight_missing=True)

escher_builder

Builder(highlight_missing=True)

## Parameterize MASS model

In [ ]:
##possible functions
# loading equilibrium constants/ fluxes, initial conditions
# potential conversion factors?
# enzyme modules?


In [ ]:
## flux split through pathways??

In [ ]:
## making enzyme modules??


#### Set fluxes

In [ ]:
## load independent fluxes
# minspan paths, independent fluxes
# steady state fluxes
# print everything

#### Set equilibrium constants

In [ ]:
# read Keq
# set Keq
# print Keqs

### Obtain Concentrations

#### Set concentrations

In [ ]:
# Provide initial guesses for a few, setting?
# load, set, print conc values

In [ ]:
## potential optimal solutions? using QP and then comparing

In [ ]:
## calculate PERC's

In [ ]:
## import/merging enzyme modules into model

## Simulate at steady state

# Export MASS model

In [16]:
save_json_mass_model(
    mass_model=glycolysis,
    filename=os.path.join(model_dir, glycolysis.id + ".json"))